# **27. LLM Trend Note 2[프로젝트]**

## **27-1. 프로젝트: KoChatGPT 업그레이드 하기**

KoChatGPT 소스코드를 바탕으로 다양한 모델 개선 전략을 선택해 KoChatGPT를 업그레이드해 보겠습니다.<br>

아래 제시된 전략 중 하나를 선택하거나 여러 개를 조합하여
여러분만의 custom ChatGPT를 개발해보세요. 물론 더 창의적인 좋은 아이디어를 도입해볼 수도 있겠죠?<br>

복수의 전략을 선택했을 때 혼자서 실험해볼 시간이 부족하다면
팀을 이뤄 분업을 해보셔도 좋습니다!<br>

> 1.우리가 지난시간 살펴본 KoChatGPT 모델에 사용한 데이터셋은 아직 완벽히 정제되지 않았습니다.

> 2.Hunman Feedback이 반영된 데이터셋을 대체하기 위해
SFT와 RM 모델에 사용할 다양한 benchmark 데이터셋도 검토해볼 수 있습니다.

> 3.언어모델의 생성능력을 좌우하는 최선의 디코딩을 위한 하이퍼파라미터 서치가 필요합니다.

> 4.생성된 답변에 대한 주관적인 평가를 보완할 수 있는 정량적인 메트릭은 도입하지 않았었습니다.

> 5.LLM Trend Note1에서 살펴본 다양한 Instruction Tuning 및 Prompting 기법들도 적용해볼만 합니다.

> 6.무엇보다 foundation model로 사용한 KoGPT-2는 Emergent abilities를 기대하기엔 다소 작은 사이즈의 모델입니다.
더 큰 파라미터 스케일을 가진 모델을 사용해보거나,

> 7.더 효율적인 연산을 수행할 수 있는 LoRA의 적용 또는
새로운 Instruction Tuning 및 reward ranking 알고리즘을 도입해볼 수도 있습니다.

어떤 걸 해야할 지 감이 잡히지 않는 분들을 위해
몇 가지 예시를 소개해드리도록 하겠습니다.

#### **기존 데이터셋 추가 정제**

data_kochatgpt 폴더에는 세 파일이 있습니다.<br>
ㄱ. kochatgpt_1_SFT.jsonl : SFT를 위한 prompt와 completion 문장셋<br>
ㄴ. kochatgpt_1_RM.jsonl : RM 학습을 위한 prompt와 세 가지 ranking 문장셋<br>
ㄷ. kochatgpt_1_PPO.jsonl : promt 문장<br>

각 말뭉치를 EDA하여 도메인과 문체, 길이분포, 문장의 완성도 등을 분석합니다.<br>
언어모델의 문장생성능력은 말뭉치의 전처리 수준에 큰 영향을 받습니다.<br>
말뭉치의 분석결과를 토대로 데이터를 정제하여 모델을 재학습시켜봅니다.<br>
(정제후 데이터셋 크기가 줄어들지 않도록, 다양한 augmentation 기법을 활용하여 크기를 유지 내지 증량합니다.)<br>
추가 전처리 후, 기존 인퍼런스 결과와 성능을 비교해봅니다.<br>
(주관적인 평가와 BLEU, ROUGE 등을 활용한 정량적인 평가 결과를 비교 분석하여 제시합니다.)

#### **새로운 데이터셋 추가**

KoChatGPT는 human feedback이 반영된 데이터를 직접 사용하는 대신<br>
ChatGPT API를 사용하는 대안을 선택했습니다.<br>
LLM Trend Note1 에서 살펴보았듯이<br>
Anthropic의 RLHF는 StackExchange 같은 온라인 상의 댓글정보를 활용하여<br>
ranking dataset을 구축해 구현되었습니다.<br>
우리도 비슷한 로직을 적용해볼 수 있습니다.<br>

하나의 prompt에 대한 다양한 수준의 품질로 댓글이 달린 한국어로 된 웹사이트를 찾아봅시다.<br>
웹크롤링 기법을 사용해 reward 점수를 차등적으로 적용해볼 수 있는<br>
instruction dataset과 ranking dataset을 구축해봅니다.<br>

[KorQuAD 2.0](https://korquad.github.io/) 같은 한국어 이해 benchmark를 활용해 고품질의 데이터셋을 확보하고,<br>
KoGPT-2를 사용해 빠르게 저품질 데이터셋을 페어링해볼 수도 있습니다.<br>
다양한 데이터 증량전략을 구사하여 기존 데이터셋에 새로 구축한 데이터셋을 추가해<br>
모델을 재학습시키고 추론 결과를 비교하 분석하여 제시해보세요.

#### **foundation model 교체**

현재 제공되는 LMS GPU 사양으로는 수십 billion 단위 이상의 LLM을 튜닝하기 어렵습니다.<br>
그러나 허깅페이스에서 제공하는 큰 규모의 모델을 적은 컴퓨팅 자원으로도 사용할 수 있게 해주는<br>
경량화, 최적화 라이브러리를 사용하면<br>
속도는 느리지만 우리의 LMS에서도 학습 및 추론이 가능해질 수 있습니다.<br>
(힌트 : LLM Trend Note1 노드의 마지막 스텝을 참고해보세요)<br>

허깅페이스에서 제공되는 1.2B 사이즈의 한국어 GPT pretrain model로 [skt/ko-gpt-trinity-1.2B-v0.5](https://huggingface.co/skt/ko-gpt-trinity-1.2B-v0.5) 가 있습니다.<br>
해당 모델로 foundation model을 교체해보세요.<br>
(단 OOM 문제를 해소하기 위해 허깅페이스에서 제공하는<br>
다양한 training argument들을 조합하여 최상의 하이퍼파라미터를 찾아내야 합니다.)<br>
데이터셋을 아예 바꿔 모델 선택의 폭을 늘려보는 것도 좋은 선택지입니다.<br>

foundation model 교체에 성공했다면, generator 함수를 수정하여 모델 인퍼런스 결과를 제시해보세요.

##### **참고**

LLM Trend Note2 노드에서 살펴본 KoChatGPT 소스코드는<br>
빠르게 baseline모델을 설계해 실습해보기 위해 오리지널 코드를 일부 수정한 버전입니다.<br>
프로젝트 진행을 위해 모델을 커스터마이징할 때, 필요시 "colossalai_ChatGPT_230319" 폴더 내의 원본 스크립트들을 참고하세요.

## **27-2. 프로젝트 제출**

##### **루브릭**
아래의 기준을 바탕으로 프로젝트를 평가합니다.

|평가문항|상세기준|
|--|--|
|1. 기존 데이터셋을 추가 정제하고, generation 성능을 끌어올리기 위한 기법들을 실험해 모델 perfomance를 향상시켜보았는가?	|기존 데이터셋의 문제점을 분석하고 전처리 전략을 수립해 추가 정제를 진행했다. Beam search, Top-k(p) sampling 등 최선의 디코딩 전략을 수립해 향상된 모델 추론 결과를 제시했다. BLEU, ROUGE 등 생성된 텍스트를 평가하기 위한 메트릭을 적용한 정량적인 평가 결과와 주관적인 평가를 비교분석하였다.|
|2. 새로운 데이터를 수집해 전처리를 수행하여 모델을 재학습시켜보았는가?|모두의 말뭉치, AI hub 등에 공개된 데이터를 사용해 추가 데이터셋을 구축하기 위한 기준과 근거를 수립했다. ChatGPT API나 다양한 한국어 benchmark 데이터셋을 활용해 Human Feedback 을 대체할 수 있는 아이디어를 구현했다. 위를 바탕으로 SFT, RM, PPO 세 단계에 필요한 각 데이터셋을 적절히 구축하여, 모델 추론 결과와 수립한 가설을 비교해보았다.|
|3. 학습 전략 또는 foundation model을 변경해 모델을 재학습시켜보았는가?|더 적절한 Instruction Tuning 기법을 적용해 SFT를 해보거나, Reward Model의 ranking algorithm을 개선해보았다. KoGPT-2가 아닌 다른 모델을 initial model로 사용하여 모델 학습을 성공시켰다. 허깅페이스의 accelerate, bitsandbytes 라이브러리 등을 사용하여 더 큰 스케일의 모델로 ChatGPT를 re-building해 모델 성능을 향상시켰다.|